# Process US Census Commuting Flows

This script adds the US Census Commuting Flow data in order to get a count of people commuting into and out of each county in the dataset.

Data obtained from uscensus.gov, available [here](https://www.census.gov/data/tables/2015/demo/metro-micro/commuting-flows-2015.html) (table 1 and table 2 in particular)

In [1]:
# change this variable to the table of processed data from 010
DATA_URL = "kaiser_processed_70days.csv"

import pandas as pd

def read_as_string(val):
    return str(val)

# import all needed data
kaiser_df = pd.read_csv(DATA_URL, index_col=0)

# this uses convertors to ensure that the imported FIPS codes keep the leading zeros
commuting_df = pd.read_csv("uscensus_commuting_flows.csv", converters={
    "res_state_fips": read_as_string,
    "res_county_fips": read_as_string,
    "dest_state_fips": read_as_string,
    "dest_county_fips": read_as_string,
})

After initialization, iterate over the existing rows and add the commuting data:

In [2]:
# store updated rows in this list
new_df_rows = []
for index, row in kaiser_df.iterrows():
    # find the rows involving this FIPS
    string_fips = f"{row.fips:05}"
    dest_rows = commuting_df.loc[(commuting_df.dest_state_fips.str[1:] == string_fips[:2]) & (commuting_df.dest_county_fips == string_fips[2:])]
    res_rows = commuting_df.loc[(commuting_df.res_state_fips == string_fips[:2]) & (commuting_df.res_county_fips == string_fips[2:])]

    # filter out commuting from this county to this county
    commuting_within = dest_rows.loc[dest_rows.dest_county_fips == dest_rows.res_county_fips]
    dest_rows = dest_rows.loc[dest_rows.dest_county_fips != dest_rows.res_county_fips]
    res_rows = res_rows.loc[res_rows.dest_county_fips != res_rows.res_county_fips]

    # sum the workers in each category and add to the row
    row["commuting_within"] = int(commuting_within.workers.sum())
    row["commuting_out"] = int(res_rows.workers.sum())
    row["commuting_in"] = int(dest_rows.workers.sum())

    new_df_rows.append(row)

Finally, save the rows as a dataframe to a csv

In [3]:
new_data_df = pd.DataFrame(new_df_rows)
new_data_df.to_csv(DATA_URL.replace("kaiser", "commuting_flows"))